# SimiCPipeline - Full Tutorial

>*Author: Irene Marín-Goñi, PhD student - ML4BM group (CIMA University of Navarra)*

This notebook provides a comprehensive guide to how to use the new SimiC pipeline with all available features.


## Overview
This tutorial covers:
1. Package installation and setup
2. Complete pipeline initialization with all parameters
3. Running the full pipeline with filtering and AUC calculation steps
4. Follow up steps: Detailed results analysis
5. Analyze previous runs
6. Cross-validation and parameter sweeps

For a simpler introduction, see `Tutorial_SimiCPipeline_simple.ipynb`

## Introduction
SimiC is a GRN inference algorithm for scRNA-Seq data that takes as input:
- single-cell imputed expression data (not sparse)
- a list of driver genes (transcription factors)
- the cell labels (cell phenotypes in order)

and produces TF-specific GRNs for each of the different phenotypes. Given the phenotype order, SimiC adds a similarity constraint when jointly inferring the GRNs for each phenotype, ensuring a smooth transition across phenotypes.

                        0 -> 1; 1 -> 2; 2 -> 3

For more information check our original publication:

Peng, J., Serrano, G., Traniello, I.M. et al. <em>SimiC enables the inference of complex gene regulatory dynamics across cell phenotypes.</em>  Commun Biol 5, 351 (2022) <b>DOI</b>: [10.1038/s42003-022-03319-7]( https://doi.org/10.1038/s42003-022-03319-7).

## Setup

The easiest way to configure your environment is to follow the `README` instructions using `poetry` (or `Docker`).


Required packages for this tutorial:
- simicpipeline
- anndata
- pandas
- numpy
- os
- pickle

Internally simicpipeline also uses:
- scipy
- sklearn

We recommend preprocessing the data and set up the directory structure following the `Tutorial_SimiCPipeline_preprocessing.ipynb`


## Import Modules

First, import the necessary modules and set up the path.

In [ ]:
import os
print(os.getcwd())
print(os.listdir())
import simicpipeline 
print(f"SimiC pipeline version: ", {simicpipeline.__version__})
from simicpipeline import SimiCPipeline

/home/workdir
['SimiCExampleRun', 'data']
SimiC pipeline version:  {'0.1.0'}


## Pipeline steps

### Step 1: Initialize the Pipeline

Create a pipeline instance by specifying:
- `project_dir`: Working directory path where input files are located and output files will be saved
- `run_name`: Unique identifier for this analysis run (used as prefix for output files)

We will start this tutorial with the output files generated in the preprocessing tutorial.

In [2]:
print("Initializing SimiC pipeline")
pipeline = SimiCPipeline(
    project_dir="./SimiCExampleRun/KPB25L/Tumor",
    run_name="experiment_tumor"
)
print("\n" + "="*70)
print(f"Pipeline initialized with workdir: {pipeline.project_dir}")
print("="*70)
pipeline.print_project_info()

Initializing SimiC pipeline

Pipeline initialized with workdir: SimiCExampleRun/KPB25L/Tumor
Tumor/
├── inputFiles/
│   ├── TF_list.csv
│   ├── expression_matrix.pickle
│   └── treatment_annotation.txt
└── outputSimic/
    ├── figures/
    └── matrices/


### Step 2: Set Input File Paths

Point the pipeline to your input files:
- `p2df`: Path to expression matrix file (genes × cells) stored as a pandas DataFrame in pickle format
- `p2tf`: Path to transcription factor list file (pickle format) containing TF gene names to use as drivers
- `p2assignment`: Path to cell cluster assignment file (.txt format) containing phenotype labels as integers matching expression matrix cell order
- `df_with_label`: Whether the dataframe contains a label column with assignment labels. (Defaut: `False`)

In [3]:
print("Setting input file paths")
pipeline.set_input_paths(
    p2df = pipeline.project_dir / "inputFiles/expression_matrix.pickle",
    p2tf = pipeline.project_dir / "inputFiles/TF_list.csv",
    p2assignment = pipeline.project_dir / "inputFiles/treatment_annotation.txt"
)

Setting input file paths


### Step 3: Set Parameters

Set all available parameters for the SimiC regression:

#### Core Regularization Parameters:
- `lambda1`: L1 regularization strength controlling sparsity in the learned networks (higher values $\approx $ sparser networks with fewer edges, default: $1e^{-1}$)
- `lambda2`: L2 regularization strength controlling similarity between adjacent phenotype networks (higher values $\approx $ more similar networks across phenotypes, default: $1e^{-5}$)
- `similarity`: Boolean flag to enable similarity-based clustering when inferring GRNs (default: `True`)

#### Algorithm Parameters:
- `max_rcd_iter`: Maximum number of iterations for the coordinate descent optimization algorithm (default: 50000)

<div class="alert alert-block alert-danger">
<b> IMPORTANT NOTE:</b> This parameter controls the number of iterations of the algorithm and will strongly influence the time it takes to run
</div>

#### Cross-Validation Parameters:
- `cross_val`: Boolean flag to enable cross-validation for automatic parameter selection (default: False)
- `k_cross_val`: Integer for the number of folds for cross-validation (default: 5)
- `max_rcd_iter_cv`: SImilar to `max_rcd_iter`. To speed up the process the default value is 10000. 
- `list_of_l1`: List of lambda1 values to test during cross-validation (e.g., [10, 1, 1e-1, 1e-2, 1e-4])
- `list_of_l2`: List of lambda2 values to test during cross-validation (e.g., [1e-1, 1e-2, 1e-3, 1e-4, 1e-5])

<div class="alert alert-block alert-danger">
<b>IMPORTANT NOTE:</b> Cross-validation with multiple lambda values is computationally intensive and may take several hours depending on data size, max_rcd_iter_cv and parameter grid complexity.
</div>

In [ ]:
print("Setting custom parameters")
pipeline.set_parameters(
    lambda1=1e-1,
    lambda2=1e-2,
    similarity=True,
    max_rcd_iter=5000, # For demonstration purposes we set a smaller number of max iterations, udually 50000
    cross_val=True,
    k_cross_val=4,
    max_rcd_iter_cv=1000, # For demonstration purposes we set a smaller number of max iterations for cross-validation, usually 5000
    list_of_l1=[1e-1, 1e-2],  # Example: [10, 1, 1e-1, 1e-2, 1e-4]
    list_of_l2=[1e-2, 1e-3] # Example: [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
)

Setting custom parameters


<a id='step4'></a>
### Step 4: Configure AUC Calculation Parameters

<div class="alert alert-block alert-warning">
Except for `adj_r2_threshold` the other parameters are rarely used but are available in SimiC-Jianhao
</div>

Define parameters for Area Under Curve calculation which measures TF activity scores:
- `adj_r2_threshold`: Minimum adjusted R² value for filtering target genes based on regression quality (range: 0-1, typical: 0.7)
- `select_top_k_targets`: Number of top-ranked targets to include per TF (None: use all targets, integer: specific number)
- `percent_of_target`: Percentage of targets to include after ranking (range: 0-1, default: 1 for 100%)
- `sort_by`: Criterion for ranking and selecting targets ('expression': by mean expression level, 'weight': by network weight, 'adj_r2': by adjusted R² value)
- `num_cores`: Number of CPU cores to use for parallel processing (-1: use all available cores, positive integer: specific number, default: 1)

In [5]:
auc_params = {
    'adj_r2_threshold': 0.7,
    'select_top_k_targets': None,
    'percent_of_target': 1,
    'sort_by': 'expression',
    'num_cores': -2
}

Validate the inputs before run the pipeline

In [6]:
pipeline.validate_inputs()

✓ All required input files found


<a id='step5'></a>

### Step 5: Run the Complete Pipeline

Executing the full pipeline in one command is easy with this implementation.

Steps incuded:
1. Input validation
2. SimiC regression
3. Weight filtering
4. AUC calculation

#### Custom Pipeline Execution Options:
- `skip_filtering`: If `True`, skips the weight filtering step that removes low-importance edges (default: False)
- `calculate_raw_auc`: If `True`, calculates AUC scores on unfiltered regression weights before any filtering (default: False)
- `calculate_filtered_auc`: If `True`, calculates AUC scores on filtered weights after removing noise (default: True)
- `variance_threshold`: Threshold for TF selection based on variance explained - keeps TFs explaining this cumulative percentage of target variance (range: 0-1, common value: 0.9 for 90%)
- `auc_params`: Dictionary containing all AUC calculation parameters as defined in [Step4](#step4)

In [8]:
print("Running complete SimiC pipeline...")
print("This may take several minutes depending on data size.\n")

pipeline.run_pipeline(
    skip_filtering=False,
    calculate_raw_auc=False,
    calculate_filtered_auc=True,
    variance_threshold=0.9,
    auc_params=auc_params
)

Running complete SimiC pipeline...
This may take several minutes depending on data size.


STARTING SIMIC PIPELINE

Running SimiC Regression
Run name: experiment_tumor
✓ All required input files found
Running cross-validation with following lambdas: [0.1, 0.01] (L1), [0.01, 0.001] (L2)

Expression matrix for regression shape =  (21490, 1100)
df test =  (4298, 1100)
test data assignment set: {0, 1, 2, 3}
df train =  (17192, 1100)
train data assignment set: {0, 1, 2, 3}
-------
-------
.... generating train set
cell type  0
	TF size: (3475, 101)
	Target size: (3475, 1000)
cell type  1
	TF size: (4754, 101)
	Target size: (4754, 1000)
cell type  2
	TF size: (3929, 101)
	Target size: (3929, 1000)
cell type  3
	TF size: (5034, 101)
	Target size: (5034, 1000)
-------
.... generating test set
cell type  0
	TF size: (917, 101)
	Target size: (917, 1000)
cell type  1
	TF size: (1141, 101)
	Target size: (1141, 1000)
cell type  2
	TF size: (1035, 101)
	Target size: (1035, 1000)
cell type  3
	TF siz

In [9]:
# pipeline._print_summary()

<div class="alert alert-block alert-success">
<b>Success!</b>
</div>

In [16]:
pipeline.print_project_info(max_depth=4)

Tumor/
├── inputFiles/
│   ├── TF_list.csv
│   ├── expression_matrix.pickle
│   └── treatment_annotation.txt
└── outputSimic/
    ├── figures/
    └── matrices/
        └── experiment_tumor/
            ├── experiment_tumor_L1_0.1_L2_0.01_simic_matrices.pickle
            ├── experiment_tumor_L1_0.1_L2_0.01_simic_matrices_filtered_BIC.pickle
            ├── experiment_tumor_L1_0.1_L2_0.01_wAUC_matrices_filtered_BIC.pickle
            └── experiment_tumor_L1_0.1_L2_0.01_wAUC_matrices_filtered_BIC_collected.csv


## How to continue? 

In this section we show a quick overview of all results generated by the pipeline run. All the important results are automatically saved but if you want to explore and save some of them in different format here are some examples on how to do that.

For advanced visualizations see the `Tutorial_SimiCPipeline_visualization`


### Check Available Results
<a id='load'></a>

If you stoped the pipeline and want to resume it later, you should re-initialize the pipeline using the same paths and lambda parameters. This way `pipeline.available_results()` will look for filenames corresponding to the specified lambda parameters and check if they exist.

In [1]:
import pandas as pd
import numpy as np
from simicpipeline import SimiCPipeline

pipeline = SimiCPipeline(
    project_dir="./SimiCExampleRun/KPB25L/Tumor",
    run_name="experiment_tumor"
)
pipeline.set_input_paths(
    p2df = pipeline.project_dir / "inputFiles/expression_matrix.pickle",
    p2assignment = pipeline.project_dir / "inputFiles/treatment_annotation.txt",
    p2tf = pipeline.project_dir / "inputFiles/TF_list.csv"
)
pipeline.set_parameters(
    lambda1=1e-1,
    lambda2=1e-2
)
pipeline._print_summary()


PIPELINE EXECUTION SUMMARY

Run name: experiment_tumor
Project directory: SimiCExampleRun/KPB25L/Tumor

Parameters:
  - Lambda1: 0.1
  - Lambda2: 0.01
  - Number of TFs: 100
  - Number of targets: 1000

Timing:

Available Results:
✓ Ws_raw
✓ Ws_filtered
✗ auc_raw
✓ auc_filtered




## Load and Inspect Results

Here we show how to access each item generated in the pipeline for your convinience (you can further analyze/explore them).

We designed a `load_results` function so you can easily access all the pipeline results.
- `result_type`: can be `'Ws_raw'`, `'Ws_filtered'`, `'auc_raw'` or `'auc_filtered'`

### Load Filtered Weights

Access the filtered weight matrices for downstream analysis.

In [2]:
print("="*70)
print("Accessing filtered weights...\n")

simic_results = pipeline.load_results('Ws_filtered')
print(f"SimiC results keys: {list(simic_results.keys())}")

weight_dic = simic_results['weight_dic']
print(f"\nWeight dictionary keys (labels): {list(weight_dic.keys())}")
print(f"Number of cell populations: {len(weight_dic)}")

# Inspect first population
first_label = list(weight_dic.keys())[0]
first_weights = weight_dic[first_label]
print(f"\nWeight matrix for label {first_label}:")
print(f"  Shape: {first_weights.shape}")
print(f"  Non-zero entries: {(first_weights != 0).sum()}")
pd.DataFrame(first_weights[0:3,0:5],index = simic_results['TF_ids'][0:3], columns=simic_results['query_targets'][0:5])


Accessing filtered weights...

SimiC results keys: ['weight_dic', 'adjusted_r_squared', 'standard_error', 'TF_ids', 'query_targets']

Weight dictionary keys (labels): [0, 1, 2, 3]
Number of cell populations: 4

Weight matrix for label 0:
  Shape: (101, 1000)
  Non-zero entries: 22239


,Malat1,Rn18s-rs5,Cp,Brinp3,Cmss1
Terf1,0.0,0.000000,0.0,0.0,0.0
Zfp808,0.0,-0.937451,0.0,0.0,0.0
Twist2,0.0,0.000000,0.0,0.0,0.0


### Analyze Weight Distribution

Generate summary statistics of the learned weights.

This function compares the sparsity of the network before and after filtering weights by BIC criterion (keeping only the top TFs that explain at least `variance_threshold` of the variance for each target gene, [Step5](#step5)).

In [3]:
pipeline.analyze_weights()


ANALYZING WEIGHT MATRICES

Label 0:
  Raw weights:
    - Sparsity: 0.00%
    - Avg non-zero TFs per target: 100.00
  Filtered weights:
    - Sparsity: 78.76%
    - Avg non-zero TFs per target: 21.24
  Reduction: 78.76% more sparse

Label 1:
  Raw weights:
    - Sparsity: 0.00%
    - Avg non-zero TFs per target: 100.00
  Filtered weights:
    - Sparsity: 78.06%
    - Avg non-zero TFs per target: 21.94
  Reduction: 78.06% more sparse

Label 2:
  Raw weights:
    - Sparsity: 0.00%
    - Avg non-zero TFs per target: 100.00
  Filtered weights:
    - Sparsity: 77.19%
    - Avg non-zero TFs per target: 22.81
  Reduction: 77.19% more sparse

Label 3:
  Raw weights:
    - Sparsity: 0.00%
    - Avg non-zero TFs per target: 100.00
  Filtered weights:
    - Sparsity: 77.85%
    - Avg non-zero TFs per target: 22.14
  Reduction: 77.85% more sparse



### Load Filtered AUC Scores

Examine the filtered AUC scores.

In [4]:
print("="*70)
print("Accessing filtered AUC scores...\n")

auc_filtered = pipeline.load_results('auc_filtered')
print(f"AUC dictionary keys (labels): {list(auc_filtered.keys())}")
print(f"Number of cell populations: {len(auc_filtered)}")
print("Shape of AUC matrix for first label (unprocessed):", auc_filtered[0].shape)
print("="*70)


Accessing filtered AUC scores...

AUC dictionary keys (labels): [0, 1, 2, 3]
Number of cell populations: 4
Shape of AUC matrix for first label (unprocessed): (21490, 100)


<div class="alert alert-block alert-warning">
Note that each key in the dictionary corresponds to a label in the data but the AUC matrices outputted have values for all the cells (original_matrix shape)
</div>

You need to subset the AUC matrix to get only the cells corresponding to that label. 


For that we have the following handy function (also used internally by `pipeline.subset_label_specific_auc()` that loads the AUC data and subsets for specific cells as shown below:

In [5]:
auc_subset = pipeline.subset_label_specific_auc( result_type = 'auc_filtered',label=0)
# you can now calculate the statistics on the subsetted AUC matrix
import numpy as np
print(f"  AUC score statistics:")
print(f"  Shape: {auc_subset.shape} (cells x TFs)")
print(f"    - Mean: {np.nanmean(auc_subset.values):.4f}")
auc_subset.head(3)

  AUC score statistics:
  Shape: (4392, 100) (cells x TFs)
    - Mean: 0.4249


,Terf1,Zfp808,Twist2,E2f7,Stat3,Pbx3,Arnt,Etv5,Bbx,Gtf2ird1,...,Ahr,Tead4,Nsd2,Zeb2,Mybl2,Foxn3,Smad6,Mecom,Ncor2,Tulp4
19_02_08__s1,0.268776,0.376701,0.583733,0.264931,0.361436,0.311357,0.395728,0.350450,0.533741,0.400361,...,0.316194,0.402785,0.326538,0.358114,0.313549,0.463423,NaN,0.434292,0.351161,0.436862
19_02_88__s1,0.221838,0.317149,0.773355,0.165218,0.458817,0.248040,0.669806,0.238652,0.560101,0.649818,...,0.503839,0.353500,0.269008,0.403814,0.246199,0.480579,NaN,0.470552,0.320230,0.439910
19_03_02__s1,0.355681,0.419877,0.796122,0.262525,0.306217,0.333806,0.355045,0.430247,0.548395,0.382674,...,0.292480,0.391573,0.322759,0.341600,0.320002,0.561846,NaN,0.432927,0.243152,0.406611


Notice in the output of [Step 5](#step5) that we have generated a file named `<id_name>_wAUC_matrices_filtered_BIC_collected.csv`. This data frame contains the activity scores already extracted for each of your phenotypes. You can easily incorporate this data into your Seurat/SingleCellExperiment/AnnData metadata and visualize it in UMAP. See `Tutorial_SImiCPipeline_visualization.ipynb` for an examples.

If you want to access all the activity scores you can just reload the data frame:

In [6]:
import pandas as pd
# Complete File name SimiCExampleRun/KPB25L/Tumor/outputSimic/matrices/experiment_tumor/experiment_tumor_L1_0.1_L2_0.01_wAUC_matrices_filtered_BIC_collected.csv
auc_subset_all = pd.read_csv(pipeline.p2auc_filtered.with_name(pipeline.p2auc_filtered.stem + "_collected.csv"), index_col=0)
print("Shape of collected AUC subset:", auc_subset_all.shape)
auc_subset_all.head(3)

Shape of collected AUC subset: (21490, 100)


,Terf1,Zfp808,Twist2,E2f7,Stat3,Pbx3,Arnt,Etv5,Bbx,Gtf2ird1,...,Ahr,Tead4,Nsd2,Zeb2,Mybl2,Foxn3,Smad6,Mecom,Ncor2,Tulp4
19_02_08__s1,0.268776,0.376701,0.583733,0.264931,0.361436,0.311357,0.395728,0.350450,0.533741,0.400361,...,0.316194,0.402785,0.326538,0.358114,0.313549,0.463423,NaN,0.434292,0.351161,0.436862
19_02_88__s1,0.221838,0.317149,0.773355,0.165218,0.458817,0.248040,0.669806,0.238652,0.560101,0.649818,...,0.503839,0.353500,0.269008,0.403814,0.246199,0.480579,NaN,0.470552,0.320230,0.439910
19_03_02__s1,0.355681,0.419877,0.796122,0.262525,0.306217,0.333806,0.355045,0.430247,0.548395,0.382674,...,0.292480,0.391573,0.322759,0.341600,0.320002,0.561846,NaN,0.432927,0.243152,0.406611


Or re-generated it like this: 

In [7]:
import pandas as pd
auc_subset_list = []
for label in [0,1,2,3]:
    auc_subset = pipeline.subset_label_specific_auc('auc_filtered', label=label)
    auc_subset_list.append(auc_subset)
auc_subset_all = pd.concat(auc_subset_list, axis=0)
print("Shape of collected AUC subset:", auc_subset_all.shape)
auc_subset_all.head(3)
# auc_subset_all.to_csv("auc_filtered_collected.csv")


Shape of collected AUC subset: (21490, 100)


,Terf1,Zfp808,Twist2,E2f7,Stat3,Pbx3,Arnt,Etv5,Bbx,Gtf2ird1,...,Ahr,Tead4,Nsd2,Zeb2,Mybl2,Foxn3,Smad6,Mecom,Ncor2,Tulp4
19_02_08__s1,0.268776,0.376701,0.583733,0.264931,0.361436,0.311357,0.395728,0.350450,0.533741,0.400361,...,0.316194,0.402785,0.326538,0.358114,0.313549,0.463423,NaN,0.434292,0.351161,0.436862
19_02_88__s1,0.221838,0.317149,0.773355,0.165218,0.458817,0.248040,0.669806,0.238652,0.560101,0.649818,...,0.503839,0.353500,0.269008,0.403814,0.246199,0.480579,NaN,0.470552,0.320230,0.439910
19_03_02__s1,0.355681,0.419877,0.796122,0.262525,0.306217,0.333806,0.355045,0.430247,0.548395,0.382674,...,0.292480,0.391573,0.322759,0.341600,0.320002,0.561846,NaN,0.432927,0.243152,0.406611


### Analyze AUC Scores

Examine the distribution of TF-target predicted activity scores in each phenotype label.

In [8]:
pipeline.analyze_auc_scores()


ANALYZING AUC SCORES

Label 0:
  Shape: (4392, 100) (cells x TFs)
  AUC score statistics:
    - Mean: 0.4249
    - Median: 0.4341
    - Std: 0.1163
    - Min: 0.0060
    - Max: 0.9748
  Top 5 TFs by average AUC:
    - Tcf12: 0.6556
    - Twist2: 0.6505
    - Rere: 0.6397
    - Runx1: 0.5716
    - Baz2b: 0.5677

Label 1:
  Shape: (5895, 100) (cells x TFs)
  AUC score statistics:
    - Mean: 0.4373
    - Median: 0.4444
    - Std: 0.1217
    - Min: 0.0268
    - Max: 0.8724
  Top 5 TFs by average AUC:
    - Baz2b: 0.6813
    - Rere: 0.6284
    - Mef2a: 0.6195
    - Zmiz1: 0.5929
    - Tcf12: 0.5867

Label 2:
  Shape: (4964, 100) (cells x TFs)
  AUC score statistics:
    - Mean: 0.4515
    - Median: 0.4594
    - Std: 0.1154
    - Min: 0.0797
    - Max: 0.9789
  Top 5 TFs by average AUC:
    - Tcf12: 0.6206
    - Rfx3: 0.6188
    - Ncor2: 0.6184
    - Tulp4: 0.6021
    - Bbx: 0.5897

Label 3:
  Shape: (6239, 100) (cells x TFs)
  AUC score statistics:
    - Mean: 0.4595
    - Median: 0.4704


### Calculate Dissimilarity Scores

Compute dissimilar regulons between phenotype populations.

#### All Labels

If all labels are selected it will calculate the min-max difference among them (independently of the label order: 0,1,2)

In [9]:
print("----> For all labels:")
MinMax_all = pipeline.calculate_dissimilarity()
print(f"\nDissimilarity matrix shape: {MinMax_all.shape}")
print(type(MinMax_all))
# MinMax_all.head()

----> For all labels:

CALCULATING DISSIMILARITY SCORES ACROSS LABELS


Calculating dissimilarity scores...

Top 10 TFs by MinMax dissimilarity score:
  Twist2: 0.6755
  Esr1: 0.6109
  Zfp950: 0.6105
  Tead1: 0.5836
  Etv6: 0.5830
  Tcf7l2: 0.5761
  Trps1: 0.5739
  Runx1: 0.5715
  Nfat5: 0.5665
  Rfx3: 0.5578

Dissimilarity matrix shape: (100, 1)
<class 'pandas.core.frame.DataFrame'>


#### Specific Labels

Calculate dissimilarity between selected labels only.
- `select_labels`: List of integer labels to include in dissimilarity calculation (e.g., [0, 3] for comparing populations 0 and 3)

In [11]:
MinMax_0_3 = pipeline.calculate_dissimilarity(select_labels=[0, 3])


CALCULATING DISSIMILARITY SCORES ACROSS LABELS

Comparing labels [0, 3].

Calculating dissimilarity scores...

Top 10 TFs by MinMax dissimilarity score:
  Runx1: 0.9748
  Tead1: 0.9577
  Esr1: 0.9478
  Rere: 0.9227
  Etv5: 0.9069
  Zfp950: 0.9008
  Trps1: 0.8733
  Nfia: 0.8658
  Twist2: 0.8596
  Ets1: 0.8573


- `result_type`: Name of the AUC results to load ('auc_raw' for unfiltered or 'auc_filtered' for filtered weights)
- `label`: Integer specifying which cell phenotype/population to extract (must match labels in assignment file)

### Extract TF-Specific Networks

Get the regulatory network for specific transcription factors.
- `TF_name`: Name of the transcription factor gene (must be present in the TF list provided to the pipeline)
- `stacked`: If `True`, returns a pandas DataFrame with GRN weights for all labels in separate columns; if `False`, returns a dictionary of separate pandas Series per label

In [12]:
# Example: Get network for Bnc2
tf_name = "Bnc2"
network = pipeline.get_TF_network(TF_name= tf_name, stacked=True)

print(f"Network for {tf_name}:")
print(f"  Shape: {network.shape}")
print(f"  Columns: {list(network.columns)}")
print(f"\nTop 10 targets:")
print(network.head(10))

Retrieving network for TF: Bnc2
Network for Bnc2:
  Shape: (810, 4)
  Columns: [0, 1, 2, 3]

Top 10 targets:
                  0         1         2         3
Malat1     1.446819  0.790871  1.488290  0.000000
Rn18s-rs5 -1.031080  0.000000  0.000000 -1.535282
Cp         1.337843  0.869207 -1.046810  0.983844
Brinp3     3.691598  4.108595  3.644130  2.728581
Cmss1     -0.887657  0.000000  0.000000 -0.923735
Pcdh7      4.440973  1.408344  1.510670  0.000000
Pnrc1      0.000000  1.328640 -1.836957  1.799101
Sema5a     3.775364  2.649741  2.105157  2.844144
Slit2      5.397186  5.111741  2.203311  0.000000
Cdk14      3.930344  4.211396  2.623368  3.668307


In [19]:
# Example: Get auc for Bnc2
tf_name = ["Bnc2"]
tf_auc = pipeline.get_TF_auc(TF_name = tf_name, stacked=False)
print(type(tf_auc),"\n")

tf_auc[0].head()


Retrieving AUC for TF: ['Bnc2']
<class 'dict'> 



,Bnc2
19_02_08__s1,0.378044
19_02_88__s1,0.404421
19_03_02__s1,0.394181
19_03_23__s1,0.397918
19_03_58__s1,0.349341


In [21]:
tf_auc_stack = pipeline.get_TF_auc(TF_name= tf_name, stacked=True)
print(type(tf_auc_stack),"\n")
print(tf_auc_stack['label'].value_counts())
tf_auc_stack.head()

Retrieving AUC for TF: ['Bnc2']
<class 'pandas.core.frame.DataFrame'> 

label
3    6239
1    5895
2    4964
0    4392
Name: count, dtype: int64


,Bnc2,label
19_02_08__s1,0.378044,0
19_02_88__s1,0.404421,0
19_03_02__s1,0.394181,0
19_03_23__s1,0.397918,0
19_03_58__s1,0.349341,0


In [22]:
tf_names = ["Bnc2", "Runx2"]
tf_auc = pipeline.get_TF_auc(TF_name= tf_names, stacked=True)
tf_auc.head()

Retrieving AUC for TF: ['Bnc2', 'Runx2']


,Bnc2,Runx2,label
19_02_08__s1,0.378044,0.422599,0
19_02_88__s1,0.404421,0.564422,0
19_03_02__s1,0.394181,0.417152,0
19_03_23__s1,0.397918,0.387091,0
19_03_58__s1,0.349341,0.470390,0


## Summary

This tutorial covered:

✓ Complete pipeline initialization with all parameters

✓ Running the full analysis with filtering and AUC calculation

✓ Loading and inspecting all result types

✓ Analyzing weights and AUC score distributions

✓ Calculating dissimilarity between cell populations

✓ Extracting TF-specific regulatory networks

## Next Steps
1. **Visualize the results**: Check `Tutorial_SimiCPipeline_visualization.ipynb`
2. **Optimize parameters**: Cross-validation help you to find optimal λ₁ and λ₂ values for your data but sometimes you need to re-assess.
3. **Explore results**: Investigate specific TFs of interest and their predicted target networks
4. **Validate findings**: Compare inferred regulatory relationships with known literature or experimental data
5. **Visualize networks**: Create network diagrams for key TFs using network visualization tools

## Additional Resources
- Data preprocessing tutorial: `Tutorial_SimiCPipeline_preprocessing.ipynb`
- Visualization tutorial: `Tutorial_SimiCPipeline_visualization.ipynb`
- Check example (non interactive) scripts: `run_preprocessing.py` and `run_simicpipeline.py`
- API documentation: Check the `SimiCPipeline` class documentation for detailed information
- Original publication: [Peng et al., Commun Biol 5, 351 (2022)](https://www.nature.com/articles/s42003-022-03319-7)
- High impact publications using SimiC algorithm:
    - [Berastegui et al.](https://www.nature.com/articles/s41467-024-49529-x), <em><b>Nat Commum</em></b>, 13, 7619 (2022)
    - [Trianello et al.](https://www.nature.com/articles/s41559-023-02090-0), <em><b>Nat Ecol Evol, 7</em></b>, 1232–1244 (2023)
    - [Serrano et al.](https://www.nature.com/articles/s41467-024-49529-x), <em><b>Nat Commum</em></b>, 15, 5272 (2024)
    - [Ariño et al.](https://www.sciencedirect.com/science/article/pii/S0168827825023803), <em><b>J of Hepatol</em></b>, 84(1):135-149, (2026)
    
   